In [ ]:
import redis
import time
import pickle,os
import datetime
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
#!/usr/bin/env python
# coding: utf-8

# Author: Jialin Liu
# Python Version: 3.7
# Redis Version: 6.0.1
# About: A simple logger based on Redis
import redis
import time
import pickle


class RedisLog():
    """
    RedisLog structure
        service_name: str, service name, eg., head-detection
        func_name: str, function name within a service, e.g., detect()
        status: str, status code, 1: ok, 0: error
        error: str, error infor for the service crash, can paste from Exception
        uuid: universal id for tracing back, sending from application level, down to base service
        timestamp: auto-generated unix timestamp whenever a log is produced

    """
    def __init__(self, sname = 'RedisLog', fname = 'NA', status = 1, error = None, uuid = 0):
        self.service_name = sname
        self.func_name = fname
        self.status = status
        self.error = error
        self.uuid = uuid
        self.timestamp = int(time.time())
    def print(self):
        print("Service Name:%s"%self.service_name)
        print("Function Name:%s"%self.func_name)
        print("Status:%s"%self.status)
        print("Error:%s"%self.error)
        print("UUID:%s"%self.uuid)
        print("Timestamp:%s"%self.timestamp)
        

In [ ]:
class Redis():
    """
    Redis Class
        serialize: serialize python objects using pickle
        set_expire: set expire on a key
        get_ttl: get expire of a key
        put: put logs into redis
        get: get logs from redis
    """
    def __init__(self, host, port, password=None):
        try:
            if password == None:
                self.redis = redis.StrictRedis(host = host,
                                    port = port)
            else:
                self.redis = redis.StrictRedis(host = host,
                                    port = port,
                                    password = password)
        except Exception as e:
            #redis can not be connected
            self.redis = None
            #user should check if redis is none or not before proceeding
            pass
    def serialize(self, objs):
        """
        objs: list of python objects
        return: list of picked objects, [] if failed
        """
        try:
            pobjs=[]
            for o in objs:
                pobjs.append(pickle.dumps(o))
            return pobjs
        except Exception as e:
            print (e)
            return []
    def set_expire(self, key, ts):
        """
        key: service name
        ts: time in seconds
        return: -1 if fail
        """
        try:
            self.redis.pexpire(key,ts*1000)
        except Exception as e:
            print (e)
            return -1
    def get_ttl(self, key):
        """
        key: service name
        return: time (seconds) before expire, -1 if fail
        """
        try:
            t = self.redis.pttl(key)
            return t/1000
        except Exception as e:
            print(e)
            return -1
    def put(self, key, values):
        """
        key: service name
        values: list of logs or a single log
        return: number of logs inserted, 0 if nothing inserted        
        """
        if isinstance(values, list):
            if(len(values) ==0):
                return 0
        else:
            if values:
                values = [values]
            else:
                # values is none
                return 0
        try:
            #push all values into redis' list tail
            #serialize first
            vobjs = self.serialize(values)
            #push all objects to redis 
            if self.redis:
                self.redis.rpush(key,*vobjs)
                return len(vobjs)
            else:
                return 0
        except Exception as e:
            #in case of expection, push a simple error log into redis
            print (e)
            rlog = RedisLog(fname = 'rpush', status = 0, error = e)
            rlog_obj = self.serialize([rlog])
            try:
                self.redis.rpush('RedisLog',rlog_obj)
            except Exception as e:
                #redis failed with best try
                print (e)
                return 0
    def get(self, key, num=None):
        """
        key: service name
        num: number of logs to get
        return: list of RedisLog or [] if none found
        """
        #get latest num logs from service key
        Logs = []
        try:
            if num != None and num >0:
                objs = self.redis.lrange(key, -num, -1)
            else:
                objs = self.redis.lrange(key,0,-1)
                #print("objs:",objs)
            for o in objs:
                Logs.append(pickle.loads(o))
            return Logs
        except Exception as e:
            #print (e)
            return []

In [ ]:
#Simple Query on mac
class RedisVis():
    def __init__(self,host=None,port=None,password=None):
        self.r = None
        #setting up Redis connection
        #Test Cluster
        host = '47.99.37.157'
        port = '6379'
        password = None
        #print('Host:%s\tPort:%s\tPass:%s'%(host,port,password))
        r = Redis(host, port, password)
        self.r = r
    def latest(self, key, num=None):
        #Test retrieving latest #num logs, default 3 logs
        if key == None:
            print('Key can not be None')
            return
        if num == None:
            num = 3
        try:
            logs = self.r.get(key,num)
            for log in logs:
                log.print()
        except Exception as e:
            print(e)
            pass
    
    def tolist(self,log):
        return [log.service_name, log.func_name, log.status,log.error,log.uuid,log.timestamp]
    def all(self,key):
        #Test retrieving all logs for service 
        try:
            logs = self.r.get(key)
            llogs = []
            for log in logs:
                #print(log)
                llogs.append(self.tolist(log))
            return llogs
        except Exception as e:
            print(e)
            return None
    def _date2ts(self,date_string):
        #date_string = "11/22/2019"
        date = datetime.datetime.strptime(date_string, "%m/%d/%Y")
        timestamp = datetime.datetime.timestamp(date)
        return timestamp
    def _log2df(self, logs):
        try:
            #print(logs)
            df = pd.DataFrame(logs, columns = ['Service', 'Function','Status','Error','UUID','Timestamp']) 
            return df
        except Exception as e:
            print(e)
            return None
    def select(self, key, start, end):
        logs = self.all(key)
        #return logs
        if(not logs or len(logs)==0):
            print('No logs found')
            return
        #print('%d logs found'%len(logs))
        start = int(self._date2ts(start))
        end = int(self._date2ts(end))
        logs_df = self._log2df(logs)
        #print(logs_df['Timestamp'])
        #print(logs_df)
        #print('start:',start)
        #print('end:',end)
        #return logs_df
        if len(logs_df)>0:
            try:
                result = logs_df.loc[(logs_df['Timestamp'] <= end) & (logs_df['Timestamp'] >= start) ] 
                print('Number of logs:%d'%len(result))
                return result
            except Exception as e:
                print('error:',e)
                return None
        else:
            print('Logs to DF failed')
    def statusplot(self, key, start, end):
        df = self.select(key,start,end)
        df['Timestamp'] = pd.to_datetime(df['Timestamp'],unit='s')
        #df.head()
        df['Status'].plot()

In [ ]:
redisviz = RedisVis()

Current Keys:
- EVENT_ANALYSIS
- api-serving
- head_detection_server
- human_detection_alarm_server

# Get Latest Logs

In [ ]:
k = 'head_detection_server'
redisviz.latest(k)

In [ ]:
redisviz.latest(k,1)

# Selelct by Date

In [257]:
k = 'head_detection_server'
df=redisviz.select(k,'04/21/2020','06/04/2020')
df.head()

Number of logs:4


,Service,Function,Status,Error,UUID,Timestamp
0,head_detection_server,DetectHead,0,called by Python client\ttimeout,8f5348de-9c86-11ea-85fc-7678cf11eb37,1590191238
1,head_detection_server,DetectHead,0,called by Python client\ttimeout,944cdecc-9c86-11ea-ae24-7678cf11eb37,1590191246
2,head_detection_server,DetectHead,0,called by Python client\ttimeout,944cbdf2-9c86-11ea-85fc-7678cf11eb37,1590191246
3,head_detection_server,DetectHead,0,called by Python client\ttimeout,d296444c-9c87-11ea-b502-7678cf11eb37,1590191780


# Plot Status

In [ ]:
redisviz.statusplot(k,'05/22/2020','05/23/2020')

In [ ]:
k1 = 'human_detection_alarm_server'
df=redisviz.select(k1,'04/21/2020','06/04/2020')
df.head()

In [ ]:
redisviz.statusplot(k1,'05/22/2020','05/23/2020')

In [ ]:
k2 = 'EVENT_ANALYSIS'
df=redisviz.select(k1,'05/22/2020','05/23/2020')
df.head()